In [1]:
import numpy as np

The seed laser passes through a crystal of length L (L=1cm). In the case that the crystal isn't pumped first,
it arrives at a waist at the end of the crystal. Now, assume it is pumped and heated, giving rise to
a thermal lensing effect. We want to find the ABCD matrix that gets us between these two cases.
We need to go backwards through the unheated crystal and then forward through the heated crystal.
The first matrix is given by
$$M_0 = \pmatrix{1 & -\frac{L}{n_0} \cr 0 & 1} $$

The second matrix ABCD matrix given n_0 and n_2

$$M_2 = \pmatrix{\cos(\gamma L) & \frac{1}{n_0 \gamma} \sin(\gamma L)\cr 
            -(n_0 \gamma) \sin(\gamma L) & \cos(\gamma L) } $$

This acts on a ray via
$$\pmatrix{ x_f \cr \theta_f } = \pmatrix{ A & B \cr C & D }\pmatrix{x_i \cr \theta_i}$$

The index of refraction is given by $$n(r) = n_0 - \frac{1}{2} n_2 r^2$$
Lengths are given in m and angles in radians.

In [69]:
n0 = 1.76
#n2=1.6e-3 #cm^-2
n2 = 16 #m^-2

L = 0.01
Mat0 = np.array( [[ 1, -L/n0], [0, 1]])
gamma = np.sqrt(n2/n0)
A = np.cos(gamma*L)
B = 1/(n0 * gamma) * np.sin(gamma*L)
C = -n0 * gamma * np.sin(gamma * L)
D = np.cos(gamma*L)
Mat2 = np.array( [[ A, B], [C, D]] )
Matdif = np.matmul(Mat0,Mat2)
print(gamma)
print(Mat)
print(Matdif)

3.0151134457776365
[[ 0.99995455  0.56817321]
 [-0.01599976  0.99995455]]
[[ 1.00045444e+00  1.72160657e-06]
 [-1.59975759e-01  9.99545489e-01]]


Now we construct the initial moment matrix $\Sigma_i$ and transform it via $M$.

In [73]:
#sigmax = 64e-6 #rms beam size [m]
sigmax = 1e-3
lam = 799.89e-9 #wavelength [m]
sigtheta = lam/(4*np.pi*sigmax) #rms divergence [rad]

sigmai = np.array( [[sigmax**2, 0], [0, sigtheta**2] ] ) 
sigmaf = np.matmul(np.matmul(Matdif, sigmai),np.transpose(Matdif))
print(sigmai)
print(sigmaf)

[[1.00000000e-06 0.00000000e+00]
 [0.00000000e+00 4.05173289e-09]]
[[ 1.00090909e-06 -1.60048451e-07]
 [-1.60048451e-07  2.96402940e-08]]


Now, we relate this to the phase of a Gaussian wavefront.
We consider a coherent Gaussian given by
$$E(x) = E_0 e^{-\frac{a+i b}{2} x^2} $$
We can relate the second moment matrix to the real parameters $a$ and $b$.
$$\Sigma_{xx} = \frac{1}{2a}$$
$$\Sigma_{x\theta} = \frac{\lambda}{4\pi} \frac{b}{a}$$
$$\Sigma_{\theta \theta} = \frac{1}{2} \left(\frac{\lambda}{2\pi}\right) \left( a+ \frac{b}{a}\right) $$

From the first of these two equations, we find
$$ b = \frac{2\pi}{\lambda} \frac{\Sigma_{x\theta}}{\Sigma_{xx}} $$

In [74]:
b = (2*np.pi)/lam * sigmaf[0,1]/sigmaf[0,0]
print(b)

-1256048.5995364413


In [75]:
alpha_micron_m2 = 6.5e-7 #This number from JVT email
alpha_meter_m2 = 1e12*alpha_micron_m2
print(alpha_meter_m2)

650000.0


In [76]:
-alpha_meter_m2/b

0.5174958996330952

This factor of 2 may come from the 1D treatment given here. x-> r